# Process GREIN Human Data

Retrieve the downloaded expression data, update gene identifiers to entrez, and curate sample IDs. The script will also identify a balanced hold-out test set to compare projection performance into learned latent spaces across algorithms.

In [1]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
random.seed(1234)

## Read Phenotype Information: skipping until phenotype data found

In [3]:
# path = os.path.join('download', 'TARGET_phenotype.gz')
# pheno_df = pd.read_table(path)

# print(pheno_df.shape)
# pheno_df.head(3)

## Read Probe Mapping Info 
(where chromosomes start and end, saved in a file in downloads)

In [4]:
# path = os.path.join('download', 'gencode.v23.annotation.gene.probemap')
# probe_map_df = pd.read_table(path)

# # Inner merge gene df to get ensembl to entrez mapping
# probe_map_df = probe_map_df.merge(gene_df, how='inner', left_on='gene', right_on='symbol')

# # Mapping to rename gene expression index
# ensembl_to_entrez = dict(zip(probe_map_df.id, probe_map_df.entrez_gene_id))

# print(probe_map_df.shape)
# probe_map_df.head(3)

## Read Gene Expression Data

In [5]:
# reading in human gene expression data

file = os.path.join('download', 'grein_count_matrix_human.pkl')
expr_df = pd.read_pickle(file)

print(expr_df.shape)
expr_df.head(4)

(27990, 1309)


,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,...,GSM2692393,GSM2692394,GSM2692395,GSM2692396,GSM2692397,GSM2692398,GSM2692399,GSM2692400,GSM2692401,GSM2692402
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,36.5392,18.2983,28.3281,8.5205,17.8816,20.3450,17.4323,11.6830,11.4493,6.7943
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,0.0000,0.0000,2.9989,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,...,152.1948,106.2851,223.5236,171.2681,188.3446,133.3678,127.3812,223.1960,165.0938,108.3981
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,...,378.9609,252.2508,307.5954,264.8942,297.0646,373.8453,272.0667,166.7782,220.1159,147.8496


In [26]:
# Clean up x and y from expr_df

# create a list of cleaned columns
cleaned_column = ['gene', 'gene_symbol']
for column in expr_df.drop(['gene', 'gene_symbol'], axis = 1).columns:
    cleaned_column.append(column.split("_")[0])

# set colums to cleaned column
expr_df.columns = cleaned_column

In [27]:
# Check that it worked

pd.set_option('display.max_columns', None) 
expr_df.head()

,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,GSM2667755,GSM2667756,GSM2667757,GSM2667758,GSM2667759,GSM2667760,GSM2667761,GSM2667762,GSM2667763,GSM2667764,GSM2667765,GSM2667766,GSM2667767,GSM2667768,GSM2667769,GSM2667770,GSM2667771,GSM2667772,GSM2667773,GSM2667774,GSM2667775,GSM2667776,GSM2667777,GSM2667778,GSM2667779,GSM2667780,GSM2667781,GSM2667782,GSM2667783,GSM2667784,GSM2667785,GSM2667786,GSM2668081,GSM2668082,GSM2668083,GSM2668084,GSM2668085,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006,GSM2670874,GSM2670875,GSM2670876,GSM2670877,GSM2670878,GSM2670879,GSM2670880,GSM2670881,GSM2670882,GSM2670883,GSM2670884,GSM2670885,GSM2670886,GSM2670887,GSM2670888,GSM2670929,GSM2670930,GSM2670931,GSM2670932,GSM2670933,GSM2670934,GSM2670935,GSM2670936,GSM2670937,GSM2670938,GSM2670939,GSM2670940,GSM2670941,GSM2670942,GSM2670943,GSM2670944,GSM2671095,GSM2671096,GSM2671097,GSM2671098,GSM2671099,GSM2671100,GSM2671101,GSM2671102,GSM2671215,GSM2671216,GSM2671217,GSM2671218,GSM2671219,GSM2671220,GSM2671221,GSM2671222,GSM2671223,GSM2671224,GSM2671225,GSM2671226,GSM2671227,GSM2671228,GSM2671229,GSM2671230,GSM2671231,GSM2671232,GSM2671233,GSM2671234,GSM2671235,GSM2671236,GSM2671237,GSM2671238,GSM2671239,GSM2671240,GSM2671241,GSM2671242,GSM2671243,GSM2671244,GSM2671245,GSM2671246,GSM2671247,GSM2671248,GSM2671249,GSM2671250,GSM2671251,GSM2671252,GSM2671253,GSM2671254,GSM2671255,GSM2671256,GSM2671257,GSM2671258,GSM2671259,GSM2671260,GSM2671261,GSM2671262,GSM2671263,GSM2671264,GSM2671265,GSM2671266,GSM2671267,GSM2671268,GSM2671269,GSM2671270,GSM2671271,GSM2671272,GSM2671273,GSM2671274,GSM2671275,GSM2671276,GSM2671277,GSM2671278,GSM2671279,GSM2671280,GSM2671281,GSM2671282,GSM2671283,GSM2671284,GSM2671285,GSM2671286,GSM2671287,GSM2671288,GSM2671289,GSM2672034,GSM2672035,GSM2672036,GSM2672037,GSM2672038,GSM2672039,GSM2672040,GSM2672041,GSM2672042,GSM2672043,GSM2672044,GSM2672045,GSM2672046,GSM2672047,GSM2672048,GSM2672049,GSM2672050,GSM2672051,GSM2672052,GSM2672053,GSM2672054,GSM2672055,GSM2672056,GSM2672057,GSM2672058,GSM2672059,GSM2672060,GSM2672061,GSM2672062,GSM2672063,GSM2672064,GSM2672065,GSM2672066,GSM2672067,GSM2672068,GSM2672069,GSM2672070,GSM2672071,GSM2672072,GSM2672073,GSM2672074,GSM2672075,GSM2672076,GSM2672077,GSM2672078,GSM2672079,GSM2672080,GSM2672081,GSM2672082,GSM2672083,GSM2672084,GSM2672085,GSM2672086,GSM2672087,GSM2672088,GSM2672089,GSM2672090,GSM2672091,GSM2672092,GSM2672093,GSM2672094,GSM2672095,GSM2672096,GSM2672097,GSM2672098,GSM2672099,GSM2672100,GSM2672101,GSM2672102,GSM2672103,GSM2672104,GSM2672105,GSM2672106,GSM2672107,GSM2672108,GSM2672109,GSM2672110,GSM2672111,GSM2672112,GSM2672113,GSM2672114,GSM2672115,GSM2672116,GSM2672117,GSM2672118,GSM2672119,GSM2672120,GSM2672121,GSM2672122,GSM2672123,GSM2672124,GSM2672125,GSM2674232,GSM2674233,GSM2674234,GSM2674235,GSM2674236,GSM2674237,GSM2674238,GSM2674239,GSM2674240,GSM2674241,GSM2674242,GSM2674243,GSM2674244,GSM2674245,GSM2674246,GSM2674606,GSM2674607,GSM2674608,GSM2674609,GSM2674610,GSM2674611,GSM2674612,GSM2674613,GSM2674614,GSM2674615,GSM2674616,GSM2674617,GSM2674618,GSM2674619,GSM2674620,GSM2674621,GSM2674622,GSM2674623,GSM2674624,GSM2674625,GSM2674626,GSM2674627,GSM2674628,GSM2674629,GSM2674630,GSM2674631,GSM2674632,GSM2674633,GSM2674634,GSM2674635,GSM2674636,GSM2674637,GSM2676359,GSM2676360,GSM2674863,GSM2674864,GSM2674865,GSM2674866,GSM2674867,GSM2674868,GSM2674869,GSM2674870,GSM2674871,GSM2674872,GSM2674873,GSM2674874,GSM2674875,GSM2674876,GSM2674877,GSM2674878,GSM2674879,GSM2674880,GSM2674881,GSM2674882,GSM2674883,GSM2674884,GSM2674885,GSM2674886,GSM2674887,GSM2674888,GSM2674889,GSM2674890,GSM2674891,GSM2674892,GSM2674893,GSM2674894,GSM2674895,GSM2674896,GSM2674897,GSM2674898,GSM2674899,GSM2676335,GSM2676336,GSM2676337,GSM2676338,GSM2676339,GSM2676340,GSM2676341,GSM2676342,GSM2676343,GSM2676344,GSM2676823,GSM2676824,GSM2676825,GSM2676826,GSM2676827,

## Process gene expression matrix

This involves updating Entrez gene ids, sorting and subsetting

In [28]:
# expr_df = (expr_df
#     .dropna(axis='rows')
#     .reindex(probe_map_df.id)
#     .rename(index=ensembl_to_entrez)
#     .rename(index=old_to_new_entrez)
#     .groupby(level=0).mean()
#     .transpose()
#     .sort_index(axis='rows')
#     .sort_index(axis='columns')
# )

expr_df.index.rename('sample_id', inplace=True)

print(expr_df.shape)
expr_df.head(2)

(27990, 1309)


,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,GSM2667755,GSM2667756,GSM2667757,GSM2667758,GSM2667759,GSM2667760,GSM2667761,GSM2667762,GSM2667763,GSM2667764,GSM2667765,GSM2667766,GSM2667767,GSM2667768,GSM2667769,GSM2667770,GSM2667771,GSM2667772,GSM2667773,GSM2667774,GSM2667775,GSM2667776,GSM2667777,GSM2667778,GSM2667779,GSM2667780,GSM2667781,GSM2667782,GSM2667783,GSM2667784,GSM2667785,GSM2667786,GSM2668081,GSM2668082,GSM2668083,GSM2668084,GSM2668085,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006,GSM2670874,GSM2670875,GSM2670876,GSM2670877,GSM2670878,GSM2670879,GSM2670880,GSM2670881,GSM2670882,GSM2670883,GSM2670884,GSM2670885,GSM2670886,GSM2670887,GSM2670888,GSM2670929,GSM2670930,GSM2670931,GSM2670932,GSM2670933,GSM2670934,GSM2670935,GSM2670936,GSM2670937,GSM2670938,GSM2670939,GSM2670940,GSM2670941,GSM2670942,GSM2670943,GSM2670944,GSM2671095,GSM2671096,GSM2671097,GSM2671098,GSM2671099,GSM2671100,GSM2671101,GSM2671102,GSM2671215,GSM2671216,GSM2671217,GSM2671218,GSM2671219,GSM2671220,GSM2671221,GSM2671222,GSM2671223,GSM2671224,GSM2671225,GSM2671226,GSM2671227,GSM2671228,GSM2671229,GSM2671230,GSM2671231,GSM2671232,GSM2671233,GSM2671234,GSM2671235,GSM2671236,GSM2671237,GSM2671238,GSM2671239,GSM2671240,GSM2671241,GSM2671242,GSM2671243,GSM2671244,GSM2671245,GSM2671246,GSM2671247,GSM2671248,GSM2671249,GSM2671250,GSM2671251,GSM2671252,GSM2671253,GSM2671254,GSM2671255,GSM2671256,GSM2671257,GSM2671258,GSM2671259,GSM2671260,GSM2671261,GSM2671262,GSM2671263,GSM2671264,GSM2671265,GSM2671266,GSM2671267,GSM2671268,GSM2671269,GSM2671270,GSM2671271,GSM2671272,GSM2671273,GSM2671274,GSM2671275,GSM2671276,GSM2671277,GSM2671278,GSM2671279,GSM2671280,GSM2671281,GSM2671282,GSM2671283,GSM2671284,GSM2671285,GSM2671286,GSM2671287,GSM2671288,GSM2671289,GSM2672034,GSM2672035,GSM2672036,GSM2672037,GSM2672038,GSM2672039,GSM2672040,GSM2672041,GSM2672042,GSM2672043,GSM2672044,GSM2672045,GSM2672046,GSM2672047,GSM2672048,GSM2672049,GSM2672050,GSM2672051,GSM2672052,GSM2672053,GSM2672054,GSM2672055,GSM2672056,GSM2672057,GSM2672058,GSM2672059,GSM2672060,GSM2672061,GSM2672062,GSM2672063,GSM2672064,GSM2672065,GSM2672066,GSM2672067,GSM2672068,GSM2672069,GSM2672070,GSM2672071,GSM2672072,GSM2672073,GSM2672074,GSM2672075,GSM2672076,GSM2672077,GSM2672078,GSM2672079,GSM2672080,GSM2672081,GSM2672082,GSM2672083,GSM2672084,GSM2672085,GSM2672086,GSM2672087,GSM2672088,GSM2672089,GSM2672090,GSM2672091,GSM2672092,GSM2672093,GSM2672094,GSM2672095,GSM2672096,GSM2672097,GSM2672098,GSM2672099,GSM2672100,GSM2672101,GSM2672102,GSM2672103,GSM2672104,GSM2672105,GSM2672106,GSM2672107,GSM2672108,GSM2672109,GSM2672110,GSM2672111,GSM2672112,GSM2672113,GSM2672114,GSM2672115,GSM2672116,GSM2672117,GSM2672118,GSM2672119,GSM2672120,GSM2672121,GSM2672122,GSM2672123,GSM2672124,GSM2672125,GSM2674232,GSM2674233,GSM2674234,GSM2674235,GSM2674236,GSM2674237,GSM2674238,GSM2674239,GSM2674240,GSM2674241,GSM2674242,GSM2674243,GSM2674244,GSM2674245,GSM2674246,GSM2674606,GSM2674607,GSM2674608,GSM2674609,GSM2674610,GSM2674611,GSM2674612,GSM2674613,GSM2674614,GSM2674615,GSM2674616,GSM2674617,GSM2674618,GSM2674619,GSM2674620,GSM2674621,GSM2674622,GSM2674623,GSM2674624,GSM2674625,GSM2674626,GSM2674627,GSM2674628,GSM2674629,GSM2674630,GSM2674631,GSM2674632,GSM2674633,GSM2674634,GSM2674635,GSM2674636,GSM2674637,GSM2676359,GSM2676360,GSM2674863,GSM2674864,GSM2674865,GSM2674866,GSM2674867,GSM2674868,GSM2674869,GSM2674870,GSM2674871,GSM2674872,GSM2674873,GSM2674874,GSM2674875,GSM2674876,GSM2674877,GSM2674878,GSM2674879,GSM2674880,GSM2674881,GSM2674882,GSM2674883,GSM2674884,GSM2674885,GSM2674886,GSM2674887,GSM2674888,GSM2674889,GSM2674890,GSM2674891,GSM2674892,GSM2674893,GSM2674894,GSM2674895,GSM2674896,GSM2674897,GSM2674898,GSM2674899,GSM2676335,GSM2676336,GSM2676337,GSM2676338,GSM2676339,GSM2676340,GSM2676341,GSM2676342,GSM2676343,GSM2676344,GSM2676823,GSM2676824,GSM2676825,GSM2676826,GSM2676827,

## Stratify Balanced Training and Testing Sets in TARGET Gene Expression

Output training and testing gene expression datasets

In [ ]:
#strat = pheno_df.set_index('sample_id').reindex(expr_df.index).primary_disease_code

In [ ]:
# cancertype_count_df = (
#     pd.DataFrame(strat.value_counts()) #not using value_counts, what number should we use here? 
#     .reset_index()
#     .rename({'index': 'cancertype', 'primary_disease_code': 'n ='}, axis='columns')
# )

# file = os.path.join('data', 'target_sample_counts.tsv') #change which file - do we have a file that works for this?
# cancertype_count_df.to_csv(file, sep='\t', index=False)

# cancertype_count_df

In [7]:
train_df, test_df = train_test_split(expr_df,
                                     test_size=0.1,
                                     random_state=123) #if no stratify defined, should just randomize on its own

In [8]:
print(train_df.shape)
test_df.shape

(25191, 389)


(2799, 389)

In [12]:
#save train dataframe to file 
train_file = os.path.join('data', 'train_grein_human_expression_matrix_processed.tsv.gz')
train_df.to_csv(train_file, sep='\t', compression='gzip', float_format='%.3g')

In [13]:
#save test dataframe to file 
test_file = os.path.join('data', 'test_grein_human_expressed_matrix_processed.tsv.gz')
test_df.to_csv(test_file, sep='\t', compression='gzip', float_format='%.3g')

## Sort genes based on median absolute deviation and output to file

In [15]:
# Determine most variably expressed genes and subset
train_df_mad = abs(train_df.drop(['gene', 'gene_symbol'], axis = 1) - train_df.drop(['gene', 'gene_symbol'], axis = 1).median()).median().reset_index()
train_df_mad.columns = ['gene_id', 'median_abs_deviation']

file = os.path.join('data', 'grein_mad_human_genes.tsv')
train_df_mad.to_csv(file, sep='\t', index=False)